# Predicción del Color RNN y Desfase Fijo en Máquinas de Impresión

En este proyecto, cada máquina de impresión tiene un desfase distinto entre el momento en que deposita la tinta sobre el material y el instante en que el sensor de color mide el resultado final. Este desfase depende de la configuración física de la máquina y puede variar en número de pasos o momentos temporales.

Para estandarizar el proceso en esta tarea, consideraremos un **desfase fijo de 4 pasos** (offset = 4), lo que significa que el color que deseamos predecir en un instante *t* se medirá realmente en *t+4*. Además, utilizaremos una **ventana de 20 pasos anteriores** para hacer la predicción.

Esto implica que nuestro modelo debe cumplir con los siguientes criterios:

- A cada instante *t*, queremos predecir el color que será medido en *t+4*.
- Si offset = 4 pasos, significa que la fila *index=t+4* registrará el color correspondiente a la impresión en *t*.

Para manejar este desfase, utilizamos el enfoque conocido como **“Direct approach con offset”**:

- No necesitamos un modelo multi-output que genere predicciones para *t+1, t+2, t+3, t+4* y luego seleccionar solo el 4º valor.
- No aplicamos un método autoregresivo que retroalimente sus propias predicciones en cada paso.
- Entrenamos un modelo (por ejemplo, LSTM) que, dada una **ventana previa de 20 pasos** *(t−19..t)*, **prediga directamente el color en *t+4***.

## Ventajas y Consideraciones
✅ **Ventaja**: Simplifica la arquitectura del modelo y permite concentrar el entrenamiento en exactamente el horizonte que nos interesa.  
⚠️ **Desventaja**: No proporciona predicciones intermedias (*t+1, t+2, t+3*), pero esto no es necesario si el sensor mide consistentemente a *t+4*.

---

## **Diagrama del Flujo de la Lámina en la Máquina de Impresión**
                     *(Flujo del material en la línea de producción)*
     ┌───────────────────────────────────>────────────────────────────────┐

     ┌────────┐                  distancia "offset"                ┌──────┐
     │        │                  (4 pasos en este caso)            │      │
     │CABEZAL │--------------------------------------------------->│SENSOR│
     │IMPRENTA│                                                    │COLOR │
     └────────┘                                                    └──────┘

1️⃣ **Cabezales de impresión**
   - En el instante *t*, la máquina deposita tinta bajo ciertas condiciones (velocidad, estado de la máquina, etc.).
   - Estas condiciones influyen en el color resultante que se medirá más adelante.

2️⃣ **Desfase físico (offset)**
   - La lámina avanza hasta que el sensor de color puede medirla.
   - No se mide inmediatamente el color en *t+1* ni en *t+2*, sino hasta el tiempo *t+4*.

3️⃣ **Decisión de modelado ("Direct offset = 4")**
   - No es necesario predecir el color en *t+1, t+2, t+3*.
   - Nos enfocamos únicamente en predecir el color cuando la tinta llegue al sensor en *t+4*.
   - Para ello, entrenamos un modelo que, dada una **ventana de 20 pasos anteriores** *(t−19..t)*, prediga directamente el color en *t+4*.

---

📌 **Conclusión**  
Cada máquina puede tener un offset distinto dependiendo de su configuración, pero para esta tarea, **lo hemos estandarizado a 4 pasos** y usaremos **una ventana de 20 pasos anteriores** como entrada del modelo. Esto permite un modelado más eficiente y consistente, facilitando la predicción precisa del color en la producción. 🚀

In [1]:
### RECORDAR CAMBIAR EL ENVIRONMENT A p310_npy1_GPUok_RNN
import pandas as pd
import os
import joblib
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers, Model, Input, regularizers
from sklearn.preprocessing import StandardScaler
import itertools

# Mirar si tensorflow usará GPU

Para este modelado de RNN con direct approach, usaré la librería Tensorflow. Para verificar que la usará:

In [2]:
print("CUDA disponible:", tf.test.is_built_with_cuda())
print("cuDNN disponible:", tf.test.is_built_with_gpu_support())


CUDA disponible: True
cuDNN disponible: True


# Variables globales

In [3]:
FIC_ENTRADA='train_test_val_scaler.joblib'
RUTA_FICHEROS = r"C:\Users\jaume\Documents\Proyecto\datos"
FIC_SALIDA='prediccion_rnn_direct.joblib'
MEJOR_MODELO='mejor_modelo_rnn_direct_approach.h5'

# Parámetros para la ventana y el horizonte
WINDOW_SIZE = 30
OFFSET = 4

# Semillas de aleatoriedad

Para que sea lo más reproducible posible. Aunque ciertas cosas como el **dropout** pueden afectar al resultado

In [4]:
SEED = 7419

# Fijar la semilla en todos los módulos relevantes
#random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

os.environ['TF_DETERMINISTIC_OPS'] = '1'  # Fuerza operaciones deterministas
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # Fuerza cuDNN determinista en GPU

# Cargamos los datos

In [5]:
# Carga de datos
os.chdir(RUTA_FICHEROS)

with open(FIC_ENTRADA, 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
df_train = data_dict['train'].copy()
df_test = data_dict['test'].copy()
df_val = data_dict['val'].copy()
scaler = data_dict['scaler']              # Contiene las normalizaciones con StandarScaler()

In [6]:
df_train.columns
df_test.columns

Index(['L_scaled', 'A_scaled', 'B_scaled', 'tiempo_en_estado_scaled',
       'velocidad_scaled', 'duracion_parada_robust_scaled', 'patron_0',
       'patron_2', 'patron_3', 'patron_4',
       ...
       'patron_88', 'patron_89', 'patron_90', 'patron_91', 'patron_92',
       'patron_93', 'patron_94', 'estado_alta', 'estado_normal',
       'estado_puesta'],
      dtype='object', length=103)

## Defino las columnas target y feature

Quitamos la columna *seq_time* ya que a efectos prácticos es un índice que no aporta valor al modelo, y anteriormente ya la hemos usado para ordenar el dataframe.

In [7]:
excluir_columnas = ["L_scaled", "A_scaled", "B_scaled", "seq_time"]
feature_cols = [col for col in df_train.columns if col not in excluir_columnas]
target_cols = ["L_scaled","A_scaled","B_scaled"]

# Crear secuencias (ventanas)

**Función create_sequences_direct_offset:**

Toma un DataFrame ordenado cronológicamente y genera secuencias de largo window_size a partir de las columnas de entrada (features). El target que se predice es la fila que se encuentra a offset pasos en el futuro respecto al final de la ventana (se selecciona la fila en la posición i + window_size + offset - 1).

**Generación de secuencias para cada subconjunto:**

Se llama a la función para los dataframes de entrenamiento (df_train), validación (df_val) y test (df_test), generando así las matrices X y Y correspondientes para cada uno.

In [8]:
def create_sequences_direct_offset(df, feature_cols, target_cols, 
                                   window_size=20, offset=4):
    """
    Crea X, Y para un enfoque "Direct" a offset pasos en el futuro.
    
    - X : (N, window_size, n_features)
    - Y : (N, n_targets)  # un único step => offset steps en el futuro
    - df: DataFrame ordenado cronológicamente (por ejemplo, 'seq_time').
    
    Parámetros:
    -----------
      df : pd.DataFrame con las columnas de features + target + orden temporal.
      feature_cols : list[str], nombres de las columnas de entrada.
      target_cols : list[str], nombres de las columnas del target (ej. [L_scaled,A_scaled,B_scaled]).
      window_size : int, cuántas filas usar de histórico como input.
      offset : int, cuántos pasos a futuro se quiere predecir.
    
    Retorno:
    --------
      X : np.ndarray de shape (N, window_size, len(feature_cols))
      Y : np.ndarray de shape (N, len(target_cols))
    """

    data_features = df[feature_cols].values
    data_target   = df[target_cols].values

    X_list, Y_list = [], []
    n = len(df)

    # Se recorre hasta n - window_size - offset + 1 para que siempre exista la secuencia completa
    for i in range(n - window_size - offset + 1):
        x_seq = data_features[i : i + window_size]
        # La fila target es la que se encuentra en: i + window_size + (offset - 1)
        y_val = data_target[i + window_size + offset - 1]
        
        X_list.append(x_seq)
        Y_list.append(y_val)
    
    X_arr = np.array(X_list, dtype=np.float32)
    Y_arr = np.array(Y_list, dtype=np.float32)
    return X_arr, Y_arr

X_train, Y_train = create_sequences_direct_offset(df_train, feature_cols, target_cols, WINDOW_SIZE, OFFSET)
X_val,   Y_val   = create_sequences_direct_offset(df_val,   feature_cols, target_cols, WINDOW_SIZE, OFFSET)
X_test,  Y_test  = create_sequences_direct_offset(df_test,  feature_cols, target_cols, WINDOW_SIZE, OFFSET)

print("X_train.shape:", X_train.shape, "Y_train.shape:", Y_train.shape)
print("X_val.shape:  ", X_val.shape,   "Y_val.shape:  ", Y_val.shape)
print("X_test.shape: ", X_test.shape,  "Y_test.shape: ", Y_test.shape)

X_train.shape: (216735, 30, 100) Y_train.shape: (216735, 3)
X_val.shape:   (27062, 30, 100) Y_val.shape:   (27062, 3)
X_test.shape:  (27062, 30, 100) Y_test.shape:  (27062, 3)


# Optimizar datos para tensorflow

## Convertir los datos a tf.float32 (Formato nativo de TensorFlow)

TensorFlow maneja mejor float32 en GPUs, ya que float64 usa más memoria sin mejorar el rendimiento.

In [9]:
# Convertir X e Y a tensores de float32
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)

X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
Y_val = tf.convert_to_tensor(Y_val, dtype=tf.float32)

X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

## Verificar el tipo de datos internos (float o int)

Si hay datos enteros (int), TensorFlow podría generar errores en la optimización. Asegúrate de que X tenga solo float32.

In [10]:
print("X_train dtype:", X_train.dtype)
print("Y_train dtype:", Y_train.dtype)


X_train dtype: <dtype: 'float32'>
Y_train dtype: <dtype: 'float32'>


In [11]:
X_train = tf.cast(X_train, dtype=tf.float32)
Y_train = tf.cast(Y_train, dtype=tf.float32)

X_val = tf.cast(X_val, dtype=tf.float32)
Y_val = tf.cast(Y_val, dtype=tf.float32)

X_test = tf.cast(X_test, dtype=tf.float32)
Y_test = tf.cast(Y_test, dtype=tf.float32)

## Usar tf.data.Dataset para mayor eficiencia

Si los datos son grandes, es mejor convertirlos en tf.data.Dataset, lo cual optimiza la carga de datos y mejora el rendimiento en la GPU.

En mi caso NO los usaré ya que de momento los datos no son muy grandes y como los agregué más tarde, tendría que introducir muchos cambios en el código.

In [12]:
# Crear datasets eficientes para entrenamiento
#train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
#val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32).prefetch(tf.data.AUTOTUNE)
#test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

# Callback para medir el tiempo por época

Mediré el tiempo de los entrenamientos.

In [13]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.times = []
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_start_time)


# Definir modelo

La función `build_rnn_direct` crea un modelo con dos capas LSTM o GRU. Al usar `return_sequences=False` en la segunda capa, se obtiene un único vector de salida. Esto no es un enfoque seq2seq (donde se generaría una secuencia completa de salida).

Esta función está preparada para recibir si queremos usar LSTM o GRU así podré luego en el grid pasarlo como parámetro también, los que recibe parámetros son:

- num_features: número de features de entrada.
- num_targets: tamaño del vector de salida.
- window_size: longitud de la secuencia de entrada.
- latent_dim: número de unidades en la capa recurrente.
- dropout: fracción de dropout a aplicar en las celdas recurrentes.
- learning_rate: tasa de aprendizaje del optimizador.
- cell_type: str, puede ser "GRU" o "LSTM". Permite elegir la célula recurrente.

## Escoger la métrica para medir el resultado (bondad)

Durante el proceso de validación con el cliente, **se ha acordado que la métrica de evaluación principal será el MSE (Mean Squared Error)**. Se ha considerado que esta función de pérdida es la más adecuada para el problema, dado que prioriza minimizar los errores grandes en la predicción de color.
La función devuelve el modelo a entrenar y las métricas de bondad. Usaré MSE, aunque también calcularé MAE para comparar. El cliente determina que MSE es suficiente. 

Otras opciones eran "Delta E", "Cosine Similarity" pero el cliente por el momento las descarta.

## Explicación del Código

Este modelo RNN sigue un **enfoque directo** (direct approach), donde la entrada es una ventana de datos históricos (`window_size`) y la salida es un **único vector de predicción**.  
**Este modelo NO es seq2seq**, ya que la salida **no es una secuencia**, sino un **vector único**.

Probé un modelo con seq2seq y aunque los resultados no fueron malos, no mejoraban mucho, complicaban el modelo y no creo que sea realmente necesario para este problema.

## Parámetros de entrada de la función

| Parámetro       | Descripción |
|----------------|------------|
| **`num_features`** | Número de variables de entrada. |
| **`num_targets`** | Dimensión del vector de salida (para predicción de colores LAB: 3). |
| **`window_size`** | Cantidad de pasos temporales usados en la entrada. |
| **`latent_dim`** | Número de neuronas en cada capa recurrente. |
| **`dropout`** | Ratio de dropout para prevenir sobreajuste. |
| **`learning_rate`** | Tasa de aprendizaje del optimizador. |
| **`cell_type`** | Tipo de celda recurrente: `"GRU"` o `"LSTM"`. |

He probado con GRU y LSTM, no se ve una diferencia muy grande así que me he quedado con LSTM por estar más probado y ser más robusto.

---

## ¿Por qué cada decisión en el modelo?

**Elección entre LSTM y GRU

- **GRU:** Más rápido y eficiente en memoria.
- **LSTM:** Más potente para secuencias largas con dependencias temporales.
- **El modelo permite seleccionar ambos** (`cell_type`).

Como he comentado y ya que he construido la función para recibir este parámetro me he quedado con LSTM.

**`return_sequences=True` en la primera capa**

- Permite que la segunda capa reciba secuencias completas.
- Mejora la representación de los datos temporales.

He probado con 1, 2 y 3 capas, la que mejores resultados me ha dado es con 2.

**`BatchNormalization()`**

- Acelera la convergencia.
- Evita saturación de gradientes.
- Mejora estabilidad en el entrenamiento.

El modelo se sobreajustaba demasiado, con este parámetro también logro que se estabilice un poco más.

**`return_sequences=False` en la segunda capa**

- La red solo devuelve **el último estado** para la predicción.
- Evita que el modelo genere una secuencia completa de salida.

**`kernel_regularizer=regularizers.l2(1e-4)`**

- Evita el sobreajuste limitando la magnitud de los pesos.

El modelo se sobreajustaba demasiado, con este parámetro también logro que se estabilice un poco más.

**Optimización con Adam (AMSGrad activado)**

- **AMSGrad** mejora la estabilidad en la actualización de gradientes.
- Reduce el problema de acumulación de momento en Adam.
- Además ayuda a la hora de reproducir el mismo entrenamiento

**Función de pérdida y métricas**

- **`loss="mse"`**: Mínimos cuadrados medios, prioriza la reducción de grandes errores.
- **`metrics=["mae"]`**: Se agrega **MAE** para comparación con MSE.
- El **MSE** es la métrica seleccionada porque a parte de ser la elección que tomó la emrpesa para esta prueba de concepto:
  - Es la más adecuada para priorizar **errores grandes** en la predicción de color.
  - Minimiza las diferencias absolutas al cuadrado, lo que penaliza más los errores grandes.
  - Es **más estable** que métricas como `Cosine Similarity` o `Delta E`, que se descartan por ahora.  



In [14]:
def build_rnn_direct(
    num_features: int, 
    num_targets: int = 3, 
    window_size: int = 20, 
    latent_dim: int = 64,
    dropout: float = 0.4, 
    learning_rate: float = 1e-4, 
    cell_type: str = "GRU"
) -> tf.keras.Model:
    """
    Construye un modelo RNN (direct approach) para predecir un único paso en el futuro.

    Este modelo **NO es seq2seq**, ya que devuelve un único vector de salida en lugar de una secuencia.

    Parámetros:
    -----------
    - `num_features` (int): Número de características de entrada.
    - `num_targets` (int): Dimensión del vector de salida (por defecto, 3 para predicción de colores LAB).
    - `window_size` (int): Tamaño de la secuencia de entrada (longitud del histórico usado para predecir).
    - `latent_dim` (int): Número de unidades en cada capa recurrente.
    - `dropout` (float): Ratio de dropout para prevenir sobreajuste en las capas recurrentes.
    - `learning_rate` (float): Tasa de aprendizaje del optimizador Adam.
    - `cell_type` (str): Tipo de célula recurrente, `"GRU"` o `"LSTM"`.

    Retorna:
    -----------
    - `tf.keras.Model`: Modelo RNN compilado con optimizador Adam (amsgrad activado).

    Notas sobre el modelo:
    - Se permite elegir entre GRU y LSTM dinámicamente.
    - Se usa **BatchNormalization** después de la primera capa recurrente para mejorar estabilidad.
    - La segunda capa recurrente **no devuelve secuencias** (`return_sequences=False`).
    - Se aplica regularización L2 para evitar sobreajuste en los pesos de la red.
    - El optimizador **Adam con AMSGrad** se usa para una convergencia más estable.
    """
    
    # 🔹 Definir la arquitectura secuencial
    model = tf.keras.Sequential([
        layers.Input(shape=(window_size, num_features))  # Entrada con formato (time_steps, features)
    ])

    # 🔹 Selección de la célula recurrente
    if cell_type.upper() == "GRU":
        rnn_layer = layers.GRU
    elif cell_type.upper() == "LSTM":
        rnn_layer = layers.LSTM
    else:
        raise ValueError("Error: 'cell_type' debe ser 'GRU' o 'LSTM'.")

    # 🔹 Primera capa recurrente: 
    # - Devuelve secuencias para permitir el paso de información a la siguiente capa.
    # - Se aplica dropout para regularización.
    model.add(rnn_layer(latent_dim, return_sequences=True, dropout=dropout))

    # 🔹 Normalización para estabilizar entrenamiento y acelerar convergencia
    model.add(layers.BatchNormalization())

    # 🔹 Segunda capa recurrente: 
    # - Devuelve solo el estado final (`return_sequences=False`).
    # - Se aplica regularización L2 para evitar sobreajuste en los pesos.
    model.add(rnn_layer(latent_dim, return_sequences=False, dropout=dropout,
                        kernel_regularizer=regularizers.l2(1e-4)))

    # 🔹 Capa densa de salida: 
    # - Predice los valores de salida (num_targets).
    model.add(layers.Dense(num_targets))

    # 🔹 Configurar el optimizador Adam con AMSGrad activado
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)

    # 🔹 Compilar el modelo con MSE como función de pérdida y MAE como métrica adicional
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

    return model

# Parámetros para grid search

Defino los parámetros a cruzar y una lista para meter los resultados.

In [15]:
grid_params = {
    "latent_dim": [32, 16],
    #"dropout": [0.4, 0.6],
    "dropout": [0.7, 0.8],
    "learning_rate": [1e-4, 1e-5],
    "batch_size": [32, 64],
    #"cell_type": ["GRU", "LSTM"]  # Probar ambas células
    "cell_type": ["LSTM"]
}

results_list = []

# Entrenar el modelo

Aquí creo un bucle `for` que:

- Itera sobre todas las combinaciones de hiperparámetros definidos en el grid.
- Defino los callbacks `EarlyStopping`, `ReduceLROnPlateau` y `TimeHistory`, este último es una clase definida por mi para poder medir el tiempo de los entrenamientos.
- Entreno el modelo con `fit`.
- Los resultados los voy guardando en una lista que luego incorporaré a un dataframe que finalmente en pasos siguientes será evaluado y guardado en un fichero *joblib*.


In [ ]:
# Calculo estas 2 variables que serán fijas
num_features = len(feature_cols)
num_targets  = len(target_cols)

for latent_dim_val, dropout_val, lr_val, batch_size_val, cell_type_val in itertools.product(
        grid_params["latent_dim"],
        grid_params["dropout"],
        grid_params["learning_rate"],
        grid_params["batch_size"],
        grid_params["cell_type"]):

    print(f"Entrenando con: latent_dim={latent_dim_val}, dropout={dropout_val}, "
          f"learning_rate={lr_val}, batch_size={batch_size_val}, cell_type={cell_type_val}")
    
    # Construir el modelo con la combinación actual
    model = build_rnn_direct(num_features, num_targets, WINDOW_SIZE,
                             latent_dim=latent_dim_val,
                             dropout=dropout_val,
                             learning_rate=lr_val,
                             cell_type=cell_type_val)
    
    # Callbacks:
    # - EarlyStopping: para detener el entrenamiento si no mejora el val_loss.
    # - ReduceLROnPlateau: para ajustar la tasa de aprendizaje si el val_loss se estanca.
    # - TimeHistory: para medir el tiempo por época.
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
    time_callback = TimeHistory()

    start_time = time.time()
    history = model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=100,
        shuffle=False, # Se procesa en orden, sin aleatorización
        batch_size=batch_size_val,
        callbacks=[early_stopping, lr_scheduler, time_callback],
        verbose=1  # Poner 1 para ver el progreso
    )
    total_training_time = time.time() - start_time
    epoch_times = time_callback.times
    epochs_ran = len(epoch_times)

    # Obtener el menor valor de val_loss (MSE) del history y su índice
    best_val_mse = min(history.history["val_loss"])
    best_epoch_idx = np.argmin(history.history["val_loss"])
    
    # Opción: evaluar el modelo (con los mejores pesos restaurados) en los conjuntos de train y val.
    train_results = model.evaluate(X_train, Y_train, verbose=0)
    val_results   = model.evaluate(X_val, Y_val, verbose=0)
    # Suponiendo que el modelo se compiló con metrics=['mae', 'mse'], los índices serán:
    # 0: loss (mse), 1: mae, 2: mse
    best_train_mse_eval = train_results[0]
    best_val_mse_eval   = val_results[0]
    
    # También obtener el mínimo valor de val_mae (por si interesa)
    best_val_mae = min(history.history["val_mae"])
    
    # Guardamos el history completo para poder graficar las curvas de aprendizaje después.
    learning_curve = history.history

    # Guardar los resultados en el diccionario
    results_list.append({
        "latent_dim": latent_dim_val,
        "dropout": dropout_val,
        "learning_rate": lr_val,
        "batch_size": batch_size_val,
        "cell_type": cell_type_val,
        "best_val_mse": best_val_mse,  # extraído del history
        "best_val_mse_epoch_idx": best_epoch_idx, # Permutación donde se encontró el best_val_mse
        "best_train_mse_eval": best_train_mse_eval,  # evaluado sobre el set de entrenamiento
        "best_val_mse_eval": best_val_mse_eval,        # evaluado sobre el set de validación
        "best_val_mae": best_val_mae,
        "epochs_ran": epochs_ran,
        "epoch_times": epoch_times,
        "total_training_time": total_training_time,
        "learning_curve": learning_curve  # Este diccionario se usará para graficar las curvas de aprendizaje
    })
    
    print(f"  -> best_val_mse (history): {best_val_mse:.6f} encontrado en la permutación: {best_epoch_idx+1} , "
          f"best_val_mse (evaluated): {best_val_mse_eval:.6f}, "
          f"best_train_mse (evaluated): {best_train_mse_eval:.6f}, "
          f"best_val_mae: {best_val_mae:.6f}, "
          f"epochs: {epochs_ran}, total_time: {total_training_time:.2f}s")

# Crear un DataFrame con los resultados del grid search
df_results = pd.DataFrame(results_list)
print("\nGrid Search Results:")
print(df_results)

Entrenando con: latent_dim=32, dropout=0.7, learning_rate=0.0001, batch_size=32, cell_type=LSTM
Epoch 1/100
6773/6773 [==============================] - 95s 13ms/step - loss: 0.7178 - mae: 0.6612 - val_loss: 0.6714 - val_mae: 0.6331 - lr: 1.0000e-04
Epoch 2/100
6773/6773 [==============================] - 91s 13ms/step - loss: 0.5432 - mae: 0.5558 - val_loss: 1.1629 - val_mae: 0.8514 - lr: 1.0000e-04
Epoch 3/100
6772/6773 [============================>.] - ETA: 0s - loss: 0.4443 - mae: 0.4995
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
6773/6773 [==============================] - 91s 13ms/step - loss: 0.4442 - mae: 0.4995 - val_loss: 1.2658 - val_mae: 0.9011 - lr: 1.0000e-04
Epoch 4/100
6773/6773 [==============================] - 92s 14ms/step - loss: 0.4116 - mae: 0.4861 - val_loss: 1.3188 - val_mae: 0.9229 - lr: 5.0000e-05
Epoch 5/100
6772/6773 [============================>.] - ETA: 0s - loss: 0.3089 - mae: 0.4117
Epoch 5: ReduceLROnPlateau reducing 

# Seleccionar los mejores hiperparámetros (según el menor MSE en validación)

In [ ]:
best_idx = df_results['best_val_mse'].idxmin()
best_params = df_results.loc[best_idx]
print("\nBest Hyperparameters Found:")
print(best_params)

# Reconstruir el mejor modelo con los hiperparámetros óptimos
best_model = build_rnn_direct(num_features, num_targets, WINDOW_SIZE,
                              latent_dim=int(best_params['latent_dim']),
                              dropout=best_params['dropout'],
                              learning_rate=best_params['learning_rate'],
                              cell_type=best_params['cell_type'])

# Entrenamiento final del mejor modelo usando train y validación

In [ ]:
final_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
final_checkpoint = ModelCheckpoint(MEJOR_MODELO, monitor='val_loss', save_best_only=True)
final_lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

history_final = best_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=50,
    shuffle=False,
    batch_size=int(best_params['batch_size']),
    callbacks=[final_early_stopping, final_checkpoint, final_lr_scheduler],
    verbose=1
)

# Cargar el mejor modelo guardado
best_model = models.load_model(MEJOR_MODELO)

# Predicción sobre el conjunto de test

In [ ]:
Y_pred = best_model.predict(X_test)
print("\nPredicciones en TEST completadas.")

# Guardar resultados y predicciones

In [ ]:
data_dict = {
    "df_results": df_results,
    "predictions_test": Y_pred,
    "windows_size": WINDOW_SIZE,
    "offset": OFFSET
}

# Guardar en un solo archivo
joblib.dump(data_dict, FIC_SALIDA)
